In [11]:
# from langchain_community.utilities import SQLDatabase


# db = SQLDatabase.from_uri("sqlite:///./data/houses_buy_test.db")
# print(db.dialect)
# print(db.get_usable_table_names())
# db.run("SELECT * FROM houses LIMIT 1;")

In [1]:
import dotenv
dotenv.load_dotenv()

True

In [13]:
# from langchain_community.agent_toolkits import create_sql_agent
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [14]:
# agent_executor.invoke(
#     "Get me houses in maastricht that are lower than 250000 eur?"
# )

In [15]:
# agent_executor.invoke(
#     "I want to get a student friendly house in Nijmegen Explain How did you decide it is student friendly?"
# )

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader


embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

loader = CSVLoader('./data/houses_buy_test.csv', source_column="descrip", metadata_columns=["house_id", "url", "city", "house_type", "building_type", "price", "price_m2", "room", "bedroom", "bathroom", "living_area", "energy_label", "zip", "address", "year_built", "house_age"])
houses = loader.load()
houses

In [12]:
houses[0].json()

'{"page_content": "descrip: DE VERMELDE PRIJS  BETREFT  BIEDEN VANAF  \\u20ac 350.000,-  k.k.  Aan de  rand  van de wijk Boschpoort, nabij het Boschstraatkwartier  en  de Maas, bieden  wij u deze  vrijstaande Rijksmonumentale  Brugwachterswoning  aan  met  een karakteristieke uitstraling. Deze Brugwachterswoning is gesitueerd bij sluis 19 van de Zuid-Willemsvaart en vormt samen met de sluis, het jaagpad, de waterduiker en de naburige sluiswachterswoning een onverbrekelijk functioneel geheel.  De woning bevat onder andere een souterrain, woonkamer, keuken, 4 slaapkamers, 2 badkamers en een tuinberging en is gelegen op een perceel van 173 m\\u00b2. Diverse voorzieningen zijn in de directe omgeving aanwezig, evenals het centrum van Maastricht, welke op loop- en fietsafstand gelegen is. Tevens zijn er ook goede verbindingen met uitvalswegen richting Eindhoven-Luik (A2) en Valkenburg-Heerlen-Aachen (A79).  Algemeen: Bouwjaar: 1965 Inhoud: 725 m\\u00b3  Gebruiksoppervlakte wonen: 142 m\\u00b

In [2]:
# from langchain_community.document_transformers.openai_functions import (
#     create_metadata_tagger,
# )
# from langchain_core.documents import Document
# from langchain_openai import ChatOpenAI

# schema = {
#     "properties": {
#         "critic": {"type": "string"},
#         "tone": {"type": "string", "enum": ["positive", "negative"]},
#         "rating": {
#         "movie_title": {"type": "string"},
#             "type": "integer",
#             "description": "The number of stars the critic rated the movie",
#         },
#     },
#     "required": ["movie_title", "critic", "tone"],
# }

# Must be an OpenAI model that supports functions
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

# document_transformer = create_metadata_tagger(metadata_schema=schema, llm=llm)

In [13]:
persist_directory = './data/vecdb/'

## Here is the nmew embeddings being used
embedding = OpenAIEmbeddings()

db = Chroma.from_documents(documents=houses, 
                            embedding=embedding,
                            persist_directory=persist_directory,
                            )

db.persist()

In [3]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

persist_directory = './data/vecdb/'

db = Chroma(persist_directory=persist_directory, 
            embedding_function=OpenAIEmbeddings())

In [19]:
query = "Student housing in Maastricht"
docs = db.similarity_search_with_score(query)
print(docs[0])

(Document(page_content="house_id: 43414815\nurl: https://www.funda.nl/koop/maastricht/appartement-43414815-aldenhofpark-1-a03/\ncity: maastricht\nhouse_type: appartement\nbuilding_type: Bestaande bouw\nprice: 475000\nprice_m2: 6250.0\nroom: 3\nbedroom: 2\nbathroom: 1\nliving_area: 76\nenergy_label: na\nzip: 6211\naddress: Aldenhofpark 1 a03\nyear_built: 1915\nhouse_age: 109\ndescrip: MAASTRICHT (Jekerkwartier) -  In  het charmante  Jekerkwartier, direct  aan het stadspark van  Maastricht  op de 2e etage  gelegen instapklaar hoekappartement met  2 slaapkamers en  balkon  en een fantastisch  uitzicht op  de  kenmerkende  historische skyline met o.a. de torens van de Sint Janskerk en de Sint Servaeskerk aan het Vrijthof in Maastricht. Vanuit hier loop je zó het bruisende centrum in terwijl je niet direct de drukte van de binnenstad ervaart.  De ligging is werkelijk op een toplocatie, het onderhoud is uitstekend, de indeling zoals je het wenst: voorzien van een ruime living met uitstekende

In [13]:
import os
# langsmith traces

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="ls__720c11f30f434eaaa6fff81cd608995d"
os.environ["LANGCHAIN_PROJECT"]="housearch"

In [38]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

retriever = db.as_retriever()
real_estate_prompt = PromptTemplate.from_template("""
Query: {query}

Listings:
{listings}

The query contains a description of the desired property features and search criteria. Please evaluate how relevant each real estate listing is to the full query on a scale of 1 to 5, where:

1 = Not relevant at all
2 = Slightly relevant 
3 = Moderately relevant
4 = Very relevant
5 = Extremely relevant

Provide a numeric rating for each listing and give a qualitative short overview of how the returned houses differ:
""")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(str(doc) for doc in docs)


rag_chain = (
    {"listings": retriever | format_docs, "query": RunnablePassthrough()}
    | real_estate_prompt
    | llm
    | StrOutputParser()
)

In [42]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    public_key="pk-lf-f268dce1-a3f7-467e-8105-fb87b2a3e19e",
    secret_key="sk-lf-bf2e21cd-7682-4a3f-9f2c-94655216cb3c",
    host="https://cloud.langfuse.com"
)

In [43]:
rag_chain.invoke("""2 yatak odalı
Maastricht universitesine yakın veya toplu taşıma ile 20dk mesafede
EUR 165000 max ücreti olan
Furnished
Paylaşima açık
öğrenci kabul eden
İlana max son 1 haftada çıkmış
Kiralık mülkler""", config={"callbacks": [langfuse_handler]})

/home/user/documents/housescrape-backend/.venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


'1st listing: \nRating: 3\nOverview: This listing is moderately relevant as it meets most of the criteria such as having 2 bedrooms, being close to public transport, and being furnished. However, it is slightly above the maximum budget specified.\n\n2nd listing:\nRating: 4\nOverview: This listing is very relevant as it meets all the criteria specified in the query, including having 2 bedrooms, being close to public transport, within budget, furnished, and available for sharing.\n\n3rd listing:\nRating: 2\nOverview: This listing is slightly relevant as it has 3 bedrooms instead of 2, but it is within budget, furnished, and located close to the city center. It does not specify if it is available for sharing.'

In [24]:
# TODO: Reading metada sometimes failes in db (as in page content has metadata)
docs = retriever.invoke(test_prompt)

In [26]:
"\n\n".join(str(doc) for doc in docs)


'page_content=\'descrip: DEZE WONING WORDT  AANGEBODEN  DOOR POOTERS  MAKELAARDIJ Sfeervol  appartement gelegen in het  centrum  van Maastricht in een  autovrij gebied tussen de  Markt het gezellige  Bassin.  Het appartement beschikt  over een  woonkamer  met  open keuken, 2 slaapkamers, badkamer, ruime zolderberging en een loggia.  Indeling:  Souterrain: Onder het complex bevindt zich een privé berging (circa 3m²).  Begane grond: Via de keurige entree met bellentableau is het complex te bereiken. Met lift of via trappenhuis ga je naar de verschillende verdiepingen.  Derde verdieping: Via de entree met tegelvloer is het appartement te bereiken. Op deze verdieping is de badkamer gesitueerd en de twee slaapkamers. De badkamer (5m²) is voorzien van een ligbad met douche, vaste wastafel en een toilet. Beide slaapkamers zijn aan de achterzijde gelegen en respectievelijk 8m² en 7m² groot.  Tweede verdieping (woonniveau) Middels een trap in de entree/hal (8m²) is deze verdieping te bereiken. 